# Vertex AI LLM with BigQuery

* Summmarize and classify text
* Perform entity recognition
* Enrich data
* Run sentiment analysis




In [1]:
%pip install --upgrade --user google-cloud-bigquery-connection google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 97.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
PROJECT_ID = "genai-learning-768"  # @param {type:"string"}
REGION = "US"  # @param {type: "string"}

DATASET_ID = "bqml_llm"
CONN_NAME = "bqml_llm_conn"
LLM_MODEL_NAME = "bqml-vertex-llm"

In [2]:
import sys

# Addtional authentication is required for Google Colab
if 'google.colab' in sys.modules:

    # Authenticate user to Google Cloud
    from google.colab import auth
    auth.authenticate_user()

In [3]:
from google.cloud import bigquery
from google.cloud import bigquery_connection_v1 as bq_connection
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

In [5]:
client = bq_connection.ConnectionServiceClient()
new_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}"
exists_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}/connections/{CONN_NAME}"
cloud_resource_properties = bq_connection.CloudResourceProperties({})

# Try to use an existing connection if one already exists. If not, create a new one.
try:
    request = client.get_connection(
        request=bq_connection.GetConnectionRequest(name=exists_conn_parent)
    )
    CONN_SERVICE_ACCOUNT = f"serviceAccount:{request.cloud_resource.service_account_id}"
except Exception:
    connection = bq_connection.types.Connection(
        {"friendly_name": CONN_NAME, "cloud_resource": cloud_resource_properties}
    )
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": new_conn_parent,
            "connection_id": CONN_NAME,
            "connection": connection,
        }
    )
    response = client.create_connection(request)
    CONN_SERVICE_ACCOUNT = (
        f"serviceAccount:{response.cloud_resource.service_account_id}"
    )
print(CONN_SERVICE_ACCOUNT)

serviceAccount:bqcx-764223820039-23hm@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [6]:
!gcloud projects get-iam-policy $PROJECT_ID --flatten="bindings[].members" --format="table(bindings.role)" --filter="bindings.members:$CONN_SERVICE_ACCOUNT"

## prepare data

In [7]:
client = bigquery.Client(project=PROJECT_ID)

dataset_id = f"""{PROJECT_ID}.{DATASET_ID}"""
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION

dataset = client.create_dataset(dataset, exists_ok=True)

print(f"Dataset {dataset.dataset_id} created.")

Dataset bqml_llm created.


In [8]:
def run_bq_query(sql: str):
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results or error, if any
    """
    try:
        query_job = client.query(sql)
        result = query_job.result()
        print(f"JOB ID: {query_job.job_id} STATUS: {query_job.state}")
        return result

    except Exception as e:
        raise Exception(str(e))

# LLM with Bigquery ML

In [9]:
sql = f"""
        CREATE OR REPLACE MODEL
        `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`
        REMOTE WITH CONNECTION
          `{PROJECT_ID}.{REGION}.{CONN_NAME}`
          OPTIONS ( remote_service_type = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1');
"""
result = run_bq_query(sql)

JOB ID: a36e695b-2437-4aeb-8a7e-a7a103f87156 STATUS: DONE


## LLM

In [14]:
PROMPT = "Describe a cat in one paragraph"
sql = f"""
SELECT
            *
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                '{PROMPT}' AS prompt
              ),
              STRUCT
              (
                1 AS temperature,
                1024 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k,
                TRUE AS flatten_json_output
              ));
"""
result = run_bq_query(sql)
result.to_dataframe()

JOB ID: bbf03eb3-46d0-45a3-a73e-690b1cb4630b STATUS: DONE


,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"A cat is a small, furry mammal that is often kept as a pet. Cats are known for their independent nature and their ability to be both affectionate and aloof. They are also skilled hunters and can be very playful. Cats come in a variety of breeds, each with its own unique characteristics. Some of the most popular breeds include the Persian, Siamese, and Maine Coon. Cats are generally very easy to care for and can make great companions for people of all ages.","{""blocked"":false,""categories"":[],""scores"":[]}",,Describe a cat in oneparagrafp


In [15]:
PROMPT = "Please categorize this BBC news article into either tech, sport, business, politics, or entertainment and return the category. Here is an example. News article: Intel has unveiled research that could mean data is soon being moved around chips at the speed of light., Category: Tech "

sql = f"""
          SELECT
            body AS article_body,
            CONCAT('{PROMPT}','News article: ', 'article_body', ', Category:') as prompt_template,
            ml_generate_text_llm_result as llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT}','News article: ', body, ', Category:') AS prompt,
                body
              FROM
                `bigquery-public-data.bbc_news.fulltext`
              LIMIT
                5),
              STRUCT(1 AS temperature, 1024 AS max_output_tokens, 0.8 AS top_p, 40 AS top_k, TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)
result.to_dataframe()

JOB ID: 37ee5428-c4b7-45cd-947d-cdd3c4575b20 STATUS: DONE


,article_body,prompt_template,llm_result
0,"The global web blog community is being called into action to lend support to two imprisoned Iranian bloggers.\n\nThe month-old Committee to Protect Bloggers' is asking those with blogs to dedicate their sites on 22 February to the ""Free Mojtaba and Arash Day"". Arash Sigarchi and Mojtaba Saminejad are both in prison in Iran. Blogs are free sites through which people publish thoughts and opinions. Iranian authorities have been clamping down on prominent sites for some time. ""I hope this day will focus people,"" Curt Hopkins, director of the Committee, told the BBC News website.\n\nThe group has a list of actions which it says bloggers can take, including writing to local Iranian embassies. The Committee has deemed Tuesday ""Free Mojtaba and Arash Day"" as part of its first campaign. It is calling on the blogsphere - the name for the worldwide community of bloggers - to do what it can to help raise awareness of the plight of Mojtaba and Arash as well as other ""cyber-dissidents"". ""If you ...","Please categorize this BBC news article into either tech, sport, business, politics, or entertainment and return the category. Here is an example. News article: Intel has unveiled research that could mean data is soon being moved around chips at the speed of light., Category: Tech News article: article_body, Category:",Politics
1,"The ""digital divide"" between rich and poor nations is narrowing fast, according to a World Bank report.\n\nThe World Bank questioned a United Nation's campaign to increase usage and access to technology in poorer nations. ""People in the developing world are getting more access at an incredible rate - far faster than... in the past,"" said the report. But a spokesman for the UN's World Summit on the Information Society said the digital divide remained very real. ""The digital divide is rapidly closing,"" the World Bank report said.\n\nHalf the world's population now has access to a fixed-line telephone, the report said, and 77% to a mobile network.\n\nThe report's figures surpass a WSIS campaign goal that calls for 50% access to telephones by 2015. The UN hopes that widening access to technology such as mobile phones and the net will help eradicate poverty. ""Developing countries are catching up with the rich world in terms of access [to mobile networks],"" the report said. ""Africa is pa...","Please categorize this BBC news article into either tech, sport, business, politics, or entertainment and return the category. Here is an example. News article: Intel has unveiled research that could mean data is soon being moved around chips at the speed of light., Category: Tech News article: article_body, Category:",Business
2,"The current slew of sports games offers unparalleled opportunities for fans who like to emulate on-field action without ever moving from the couch.\n\nThe two giants in the field - ESPN and EA Sports - have been locked in a heavyweight battle for years. The latter is the world's largest games manufacturer. Years of experience mean that the titles in their steady flow of sport sims are finely honed, massively entertaining and ooze flair. Sports broadcaster ESPN, meanwhile, has leant its name to a series of games that are similarly classy but lower in profile and price. But that status quo was changed forever - or for the next 15 years at least - by a deal earlier this year when ESPN sold EA the rights to its TV branding and on-air talent, meaning the ESPN games presently developed by Sega will have to come to and end in their present form.\n\nIt was a massive-money deal that not only raised eyebrows but stirred active indignation in many quarters, with fans concerned that it is set ...","Please categorize this BBC news article into either tech, sport, business, politics, or entertainment and return the category. Here is an example. News article: Intel has unveiled research that could mean data is soon being moved around chips at the speed of light.,

* 분류
* 요약
* 감정
* 컨텐츠 생성

이런것들을 BQML로 할 수 있다

In [16]:
PROMPT = "Please return a bullet-point list of all sentences in this article that cite a statistic: "

sql = f"""
          SELECT
            body AS article_body,
            CONCAT('{PROMPT}', 'article_body') AS prompt,
            ml_generate_text_llm_result AS llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT}', body) AS prompt,
                body
              FROM
                `bigquery-public-data.bbc_news.fulltext`
              LIMIT
                5),
              STRUCT(1 AS temperature, 1024 AS max_output_tokens, 0.8 AS top_p, 40 AS top_k, TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)
result.to_dataframe()

JOB ID: 7e1fc2ce-b527-4417-9bdc-9e561980054c STATUS: DONE


,article_body,prompt,llm_result
0,"The global web blog community is being called into action to lend support to two imprisoned Iranian bloggers.\n\nThe month-old Committee to Protect Bloggers' is asking those with blogs to dedicate their sites on 22 February to the ""Free Mojtaba and Arash Day"". Arash Sigarchi and Mojtaba Saminejad are both in prison in Iran. Blogs are free sites through which people publish thoughts and opinions. Iranian authorities have been clamping down on prominent sites for some time. ""I hope this day will focus people,"" Curt Hopkins, director of the Committee, told the BBC News website.\n\nThe group has a list of actions which it says bloggers can take, including writing to local Iranian embassies. The Committee has deemed Tuesday ""Free Mojtaba and Arash Day"" as part of its first campaign. It is calling on the blogsphere - the name for the worldwide community of bloggers - to do what it can to help raise awareness of the plight of Mojtaba and Arash as well as other ""cyber-dissidents"". ""If you ...",Please return a bullet-point list of all sentences in this article that cite a statistic: article_body,"- Technorati, a blog search engine, tracks about six million blogs and says that more than 12,000 are added daily. \n\n\n- A blog is created every 5.8 seconds, according to a US research think-tank."
1,"The ""digital divide"" between rich and poor nations is narrowing fast, according to a World Bank report.\n\nThe World Bank questioned a United Nation's campaign to increase usage and access to technology in poorer nations. ""People in the developing world are getting more access at an incredible rate - far faster than... in the past,"" said the report. But a spokesman for the UN's World Summit on the Information Society said the digital divide remained very real. ""The digital divide is rapidly closing,"" the World Bank report said.\n\nHalf the world's population now has access to a fixed-line telephone, the report said, and 77% to a mobile network.\n\nThe report's figures surpass a WSIS campaign goal that calls for 50% access to telephones by 2015. The UN hopes that widening access to technology such as mobile phones and the net will help eradicate poverty. ""Developing countries are catching up with the rich world in terms of access [to mobile networks],"" the report said. ""Africa is pa...",Please return a bullet-point list of all sentences in this article that cite a statistic: article_body,"- Half the world's population now has access to a fixed-line telephone, the report said, and 77% to a mobile network.\n\n\n- The report's figures surpass a WSIS campaign goal that calls for 50% access to telephones by 2015.\n\n\n- Sixty percent of resources collected by the fund will be made available for projects in least developed countries, 30% for projects in developing countries, and 10% for projects in developed countries."
2,"The current slew of sports games offers unparalleled opportunities for fans who like to emulate on-field action without ever moving from the couch.\n\nThe two giants in the field - ESPN and EA Sports - have been locked in a heavyweight battle for years. The latter is the world's largest games manufacturer. Years of experience mean that the titles in their steady flow of sport sims are finely honed, massively entertaining and ooze flair. Sports broadcaster ESPN, meanwhile, has leant its name to a series of games that are similarly classy but lower in profile and price. But that status quo was changed forever - or for the next 15 years at least - by a deal earlier this year when ESPN sold EA the rights to its TV branding and on-air talent, meaning the ESPN games presently developed by Sega will have to come to and end in their present form.\n\nIt was a massive-money deal that not only raised eyebrows but stirred active indignation in many quarters, with fans concerned that it is set ...",Please return a bullet-point list of all sentences in this article that cite a statistic: articl